### Regression with top team against Bottom Team

In [1]:
import numpy as np
import pandas as pd
from statsbombpy import sb
pd.set_option('display.max_columns', None)
from mplsoccer import Pitch, Sbopen
from mplsoccer import VerticalPitch,Pitch
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import math 


c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.fb5ae2tyxyh2ijrdkgdgq3xbklktf43h.gfortran-win_amd64.dll
c:\Users\Barre\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G

In [3]:
parser = Sbopen()
df_competitions = parser.competition()
df_competitions

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
62,55,43,Europe,UEFA Euro,male,False,True,2020,2023-02-24T21:26:47.128979,2023-04-27T22:38:34.970148,2023-04-27T22:38:34.970148,2023-02-24T21:26:47.128979
63,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2023-06-18T19:28:39.443883,2021-06-13T16:17:31.694,None,2023-06-18T19:28:39.443883
64,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2023-07-17T21:19:03.032991,2023-07-17T21:21:56.497106,2023-07-17T21:21:56.497106,2023-07-17T21:19:03.032991
65,72,107,International,Women's World Cup,female,False,True,2023,2023-09-01T12:34:19.705316,2023-09-01T12:35:45.762196,2023-09-01T12:35:45.762196,2023-09-01T12:34:19.705316


In [4]:
teams = parser.match(competition_id=7, season_id=27)


In [11]:
teams.head(30)

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_id,country_name,competition_name,season_id,season_name,home_team_id,home_team_name,home_team_gender,home_team_group,home_team_country_id,home_team_country_name,home_team_managers_id,home_team_managers_name,home_team_managers_nickname,home_team_managers_dob,home_team_managers_country_id,home_team_managers_country_name,away_team_id,away_team_name,away_team_gender,away_team_group,away_team_country_id,away_team_country_name,away_team_managers_id,away_team_managers_name,away_team_managers_nickname,away_team_managers_dob,away_team_managers_country_id,away_team_managers_country_name,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,3901178,2016-03-12,2016-03-12 18:00:00,1,1,available,unscheduled,2023-08-05 14:29:35.200486,NaT,30,7,France,Ligue 1,27,2015/2016,165,Lorient,male,None,78,France,2983.0,Sylvain Ripoll,Sylvain Ripoll,1971-08-15,78.0,France,147,Marseille,male,None,78,France,505.0,José Miguel González Martín del Campo,Míchel,1963-03-23,214.0,Spain,1.1.0,2,2,1,Regular Season,204.0,Stade Yves Allainmat - Le Moustoir,78.0,France,78.0,Ruddy Buquet,78.0,France
1,3901260,2016-05-14,2016-05-14 21:00:00,1,0,available,unscheduled,2023-08-03 18:34:31.678988,NaT,38,7,France,Ligue 1,27,2015/2016,165,Lorient,male,None,78,France,2983.0,Sylvain Ripoll,Sylvain Ripoll,1971-08-15,78.0,France,160,Gazélec Ajaccio,male,None,78,France,121.0,Thierry Laurey,Thierry Laurey,1964-02-17,78.0,France,1.1.0,2,2,1,Regular Season,204.0,Stade Yves Allainmat - Le Moustoir,78.0,France,92.0,Benoît Millot,78.0,France
2,3829470,2015-09-20,2015-09-20 17:00:00,2,0,available,unscheduled,2023-08-03 12:36:31.634292,NaT,6,7,France,Ligue 1,27,2015/2016,135,Saint-Étienne,male,None,78,France,281.0,Christophe Galtier,Christophe Galtier,1966-08-23,78.0,France,144,Nantes,male,None,78,France,438.0,Michel Der Zakarian,Michel Der Zakarian,1963-02-18,12.0,Armenia,1.1.0,2,2,1,Regular Season,4810.0,Stade Geoffroy-Guichard,78.0,France,92.0,Benoît Millot,78.0,France
3,3901259,2016-05-14,2016-05-14 21:00:00,1,1,available,unscheduled,2023-08-06 10:27:05.199641,NaT,38,7,France,Ligue 1,27,2015/2016,133,Troyes,male,None,78,France,1003442.0,Mohamed Bradja,Mohamed Bradja,1969-11-16,4.0,Algeria,147,Marseille,male,None,78,France,3004.0,Franck Passi,Franck Passi,1966-03-28,78.0,France,1.1.0,2,2,1,Regular Season,176.0,Stade de l''Aube,78.0,France,79.0,François Letexier,78.0,France
4,3901263,2016-05-14,2016-05-14 21:00:00,1,2,available,unscheduled,2023-08-06 22:42:28.958420,NaT,38,7,France,Ligue 1,27,2015/2016,134,Rennes,male,None,78,France,1001260.0,Rolland Courbis,Rolland Courbis,1953-08-12,78.0,France,2736,Bastia,male,None,78,France,3632.0,François Ciccolini,François Ciccolini,1962-06-03,78.0,France,1.1.0,2,2,1,Regular Season,NaN,NaN,NaN,NaN,83.0,Mikael Lesage,78.0,France
5,3900594,2016-01-23,2016-01-23 21:00:00,2,2,available,unscheduled,2023-07-29 17:44:49.640408,NaT,22,7,France,Ligue 1,27,2015/2016,144,Nantes,male,None,78,France,438.0,Michel Der Zakarian,Michel Der Zakarian,1963-02-18,12.0,Armenia,146,Bordeaux,male,None,78,France,752.0,Willy Sagnol,Willy Sagnol,1977-03-18,78.0,France,1.1.0,2,2,1,Regular Season,161.0,Stade de la Beaujoire - Louis Fonteneau,78.0,France,97.0,Tony Chapron,78.0,France
6,3900587,2016-01-17,2016-01-17 22:00:00,1,0,available,unscheduled,2023-07-30 11:52:01.426143,NaT,21,7,France,Ligue 1,27,2015/2016,135,Saint-Étienne,male,None,78,France,281.0,Christophe Galtier,Christophe Galtier,1966-08-23,78.0,France,137,Lyon,male,None,78,France,283.0,Bruno Génésio,Bruno Génésio,1966-09-01,78.0,France,1.1.0,2,2,1,Regular Season,4810.0,Stade Geoffroy-Guichard,78.0,France,76.0,Clément Turpin,78.0,France
7,3900541,2015-12-05,2015-12-05 21:00:00,1,2,

In [13]:
def pass_data_monaco(match_id):
    df, related, freeze, tactics = parser.event(match_id)
    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "Paris Saint-Germain"].iloc[0]["index"]

    mask_monaco = (df.type_name == 'Pass') & (df.team_name == "Paris Saint-Germain") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_monaco, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass  



In [6]:
def pass_data_lyon(match_id):
    df, related, freeze, tactics = parser.event(match_id)

    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "Nantes"].iloc[0]["index"]
    mask_lyon = (df.type_name == 'Pass') & (df.team_name == "Nantes") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_lyon, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass

In [7]:
def avg(dictionary):
    total = sum(dictionary.values())

    average = total / len(dictionary)
    
    return average

In [14]:


def get_data(dat):
    data = pd.DataFrame()
    for index, row in dat.iterrows():
        if (row['home_team_name'] == 'Paris Saint-Germain'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            
            G = digraph(passing_data_monaco)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            match_data = {
                'Match ID': match_id,
                'Team': 'PSG',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'Paris-Saint Germain'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            G = digraph(passing_data_monaco)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
        
            match_data = {
                'Match ID': match_id,
                'Team': 'PSG',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

        if (row['home_team_name'] == 'Nantes'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            match_data = {
                'Match ID': match_id,
                'Team': 'Nantes',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'Nantes'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            match_data = {
                'Match ID': match_id,
                'Team': 'Nantes',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

    return data

d = get_data(dat=teams)
d


C:\Users\Barre\AppData\Local\Temp/ipykernel_20436/1886427101.py:81: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_index=True)
C:\Users\Barre\AppData\Local\Temp/ipykernel_20436/1886427101.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_index=True)
C:\Users\Barre\AppData\Local\Temp/ipykernel_20436/1886427101.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_index=True)
C:\Users\Barre\AppData\Local\Temp/ipykernel_20436/1886427101.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(match_data, ignore_

,Match ID,Team,Betweenness Centrality,Eigenvector Centrality,Katz Centrality,Page Rank,Average Clustering
0,3829470,Nantes,0.032323,0.395729,0.373816,0.135916,0.734073
1,3900594,Nantes,0.035354,0.416988,0.385587,0.148420,0.709964
2,3900591,PSG,0.022222,0.354768,0.348009,0.139879,0.830614
3,3901182,Nantes,0.036364,0.392293,0.365248,0.123703,0.684078
4,3901170,Nantes,0.016162,0.344772,0.339402,0.136020,0.863289
5,3901166,PSG,0.018182,0.346859,0.341401,0.149594,0.857641
6,3901261,PSG,0.015278,0.353185,0.346445,0.152346,0.893220
7,3901261,Nantes,0.031313,0.357399,0.346368,0.153602,0.749959
8,3901246,Nantes,0.024242,0.368780,0.356606,0.158064,0.801864
9,3901236,Nantes,0.090909,0.468032,0.375852,0.185618,0.461913


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [24]:
team_counts = d['Team']
team_counts
print("Number of Lyon teams:", team_counts.get("Paris Saint-Germain", 0))
print("Number of AS Monaco teams:", team_counts.get("Nantes", 0))

Number of Lyon teams: 0
Number of AS Monaco teams: 0


In [39]:
X = d.drop(['Team','Match ID'], axis=1)
y = d['Team']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.22, random_state=30)
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [35]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [36]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy: 53.85%
Confusion Matrix:
 [[7 0]
 [6 0]]
Classification Report:
               precision    recall  f1-score   support

      Nantes       0.54      1.00      0.70         7
         PSG       0.00      0.00      0.00         6

    accuracy                           0.54        13
   macro avg       0.27      0.50      0.35        13
weighted avg       0.29      0.54      0.38        13



In [41]:
base_classifier = DecisionTreeClassifier(max_depth=6)  # You can adjust max_depth as needed

model = AdaBoostClassifier(base_classifier, n_estimators=20, random_state=33)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9230769230769231
